In [1]:
if not('init' in globals()):
    import os
    os.chdir('../..')

    print("Current working directory: {0}".format(os.getcwd()))

    import torch 
    import torch.nn as nn

    from tests.utils import *

    import model.convolution
    import model.Quantizer
    import model.Conversion

    init = 0


Current working directory: /home/dschnoell/sources/yolov3/EfficientConvBNQuantization


In [2]:
dev = "cpu"
a = torch.randn(10,5,30,30).to(dev)
a = a.clamp(-0.5,0.5)

In [3]:
a2 = a.detach().clone()
ao = a.detach().clone()
exp = -8*torch.ones((1,5,1,1)).to(dev)
a2 = (a2/(2**(exp))).type(torch.int64)
# print(a2)


fun = lambda a:torch.ones_like(a)
fun = None

In [4]:
bias = True

Conv = nn.Conv2d(5,5,3,1,padding=1,bias=bias).to(dev)
Convq = model.convolution.Conv2d(5,5,3,1,padding=1,bias=bias,weight_quant_bits=8,weight_quant_channel_wise=True,out_quant_args=[8,(-1,)]).to(dev)

inquant = model.Conversion.Start().to(dev)
outquant = model.Conversion.Stop().to(dev)

Convq.weight.data = Conv.weight.detach().clone() 
if bias:
    Convq.bias.data = Conv.bias.detach().clone() 

In [5]:
a = ao.clone()
Convq.train()
outquant.train()
inquant.train()


a = inquant(ao.clone())
out2 = Convq(a,fun)

if bias:
    pr("quant bias", Conv.bias, Convq.debug_bias)
    pr("quant bias check", Conv.bias, Convq.bias)


pr("quant weights", Conv.weight, Convq.debug_weight)
pr("quant weights check", Conv.weight, Convq.weight)

quant bias diff: 7.486902177333832e-06, mean: 5.252659320831299e-06
quant bias check diff: 0.0, mean: 0.0
quant weights diff: 0.0017336388118565083, mean: 0.0006302772089838982
quant weights check diff: 0.0, mean: 0.0


In [6]:
a = ao.clone()
Convq.train()
outquant.train()
inquant.train()





a = inquant(ao.clone())
pr("a start:",a,inquant(ao.clone()))
# ao = a.clone()
out1 = Conv(ao.clone())
# out1 = outquant(out1)
out2 = Convq(a,fun)
pr("a",a,inquant(ao.clone()))
pr("out", out1,out2.get()[0])
pr("weights", Convq.weight,Conv.weight )
pr("quant weights", Conv.weight, Convq.debug_weight)
if bias:
    pr("quant bias", Conv.bias, Convq.debug_bias)

Conv.weight.data = Convq.debug_weight.clone()
if bias:
    Conv.bias.data = Convq.debug_bias.clone()

# print("outquant.delta_out",outquant.delta_out.view(-1))
print("out quant",out2.get()[1].exp2().view(-1))
# pr("out quant delta",outquant.delta_out.view(-1),out2.get()[1].exp2().view(-1))


out2_weight = Convq.debug_weight.clone()
out2_weight_fact = Convq.debug_fact.clone()
out2_weight_delta = Convq.weight_quant.delta_out.clone()
# out2_bias = Convq.bias.clone()



Convq.eval()
outquant.eval()
inquant.eval()
b = inquant(ao.clone())
# b = a
out3 = Convq(b,fun)
out3v = (out3.get()[0].type(torch.float))*(out3.get()[1].exp2())
# print(out3[0])
pr("out", out1,out3v)
pr("out bits", (out1/(2**out3.get()[1])),out3.get()[0])
pr("eval cmp", (out2.get()[0]/(2**out3.get()[1])),out3.get()[0])
pr("weight quant",out2_weight.div(out2_weight_delta)*out2_weight_fact,Convq.debug_weight)


Convq.train();
print(Convq.out_quant.delta_out.view(-1))
print(Convq.debug_fact.view(-1))
print(Convq.debug_fact.log2().view(-1))
print(Convq.n.view(-1))
print(Convq.debug_n.view(-1))
if bias:
    print(Convq.t.view(-1))
    print(Convq.bias.view(-1))


a start: diff: 0.0, mean: 0.0
a diff: 0.0, mean: 0.0
out diff: 0.02267986536026001, mean: 0.00795560609549284
weights diff: 0.0, mean: 0.0
quant weights diff: 0.0017260080203413963, mean: 0.000620129460003227
quant bias diff: 5.9586018323898315e-06, mean: 2.8291717626416357e-06
out quant tensor([0.0157])
reduce autorun by 1: 1 min/max tensor([-0.5000]) / tensor([0.5000])
out diff: 0.02267986536026001, mean: 0.00795342493802309
out bits diff: 1.4458427429199219, mean: 0.5070308446884155
eval cmp diff: 1.0, mean: 0.00015555555000901222
weight quant diff: 3.814697265625e-06, mean: 4.3869019350495364e-07
tensor([0.0157])
tensor([0.5983, 0.5710, 0.5794, 0.5730, 0.6007])
tensor([-0.7411, -0.8084, -0.7875, -0.8033, -0.7354])
tensor([-11., -11., -11., -11., -11.])
tensor([-11., -11., -11., -11., -11.])
tensor([-2345.,  5626.,  1995.,  1326., 18280.])
tensor([-0.0180,  0.0431,  0.0153,  0.0102,  0.1400], grad_fn=<ViewBackward0>)
